In [1]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
import numpy as np
import tensorflow as tf 

In [8]:
data_path = 'por-eng/por.txt'
embedding_size = 256

lines = open(data_path, 'r', encoding='utf-8').read().strip().split('\n')[:10000]
pairs = [l.split('\t') for l in lines]

sources = [pair[0] for pair in pairs]
targets = [pair[1] for pair in pairs]

vocab_sources = sorted(set([word for source in sources for word in source.split()]))
vocab_targets = sorted(set([word for target in targets for word in target.split()]))

sourcer_max_len = max([len(source.split()) for source in sources])
target_max_len = max([len(target.split()) for target in targets])

In [9]:
encoder_input_data = np.zeros((len(pairs), sourcer_max_len))
decoder_input_data = np.zeros((len(pairs), target_max_len))
decoder_target_data = np.zeros((len(pairs), target_max_len, len(vocab_targets)))

for i, pair in enumerate(pairs):
    
    for j, word in enumerate(pair[0].split()):
        encoder_input_data[i, j] = vocab_sources.index(word)
        
    for j, word in enumerate(pair[1].split()):
        encoder_input_data[i, j] = vocab_targets.index(word)
        if j > 0:
            encoder_input_data[i, j - 1, vocab_targets.index(word)] = 1

ValueError: 'Go.' is not in list

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
x = Embedding(len(vocab_sources), embedding_size)(encoder_inputs)
x, state_h, state_c = LSTM(embedding_size, return_state=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(len(vocab_targets), embedding_size)(decoder_inputs)
x = LSTM(embedding_size, return_sequences=True)(x, initial_state=encoder_states)
decoder_outputs = Dense(len(vocab_targets), activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, epochs=100, validation_split=0.2)

/home/gabriel/.pyenv/versions/3.6.5/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/100
